In [1]:
#import all the necessary libraries
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from collections import Counter
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score


In [3]:
#read the raw data from local machine
data = pd.read_csv("C:\\Users\\tejan\\Downloads\\creditcardfraud\\creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [8]:
#Break the data according to class labels, randomly split them into train, test data at 75:25 proportion and join the data accordingly
data_class0 = data[data.Class == 0]
data_class1 = data[data.Class == 1]

#holdout method splitting
data_class0_train, data_class0_test = train_test_split(data_class0, test_size = 0.25,random_state = 12356) 
data_class1_train, data_class1_test = train_test_split(data_class1, test_size = 0.25,random_state = 12356) 

#joining back
data_train = pd.concat([data_class0_train, data_class1_train], ignore_index = True)
data_test = pd.concat([data_class0_test, data_class1_test], ignore_index = True)

In [9]:
#dropping the 'time' variable
data.drop(['Time'], axis = 1, inplace = True)
data_train.drop(['Time'], axis = 1, inplace = True)
data_test.drop(['Time'], axis = 1, inplace = True)

#converting the data to features and labels, then convert to arrays
X_train = data_train.iloc[:, :-1].values
y_train = data_train.iloc[:, -1].values
X_test = data_test.iloc[:, :-1].values
y_test = data_test.iloc[:, -1].values

#scale the data using by doing standardization
scaler = preprocessing.StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)
X_test_standardized = scaler.fit_transform(X_test)

In [10]:
#perform random undersampling of the data
rus = RandomUnderSampler(random_state=123)
X_rus, y_rus = rus.fit_resample(X_train_standardized, y_train)
print('Undersampled training dataset shape %s' % Counter(y_rus))
print('Original training dataset shape %s' % Counter(y_train))
print('Original testing dataset shape %s' % Counter(y_test))

Undersampled training dataset shape Counter({0: 369, 1: 369})
Original training dataset shape Counter({0: 213236, 1: 369})
Original testing dataset shape Counter({0: 71079, 1: 123})


In [24]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#build a SVM model using sigmoid activation function and C = 0.00001
svm_clf = SVC(kernel='sigmoid', C = 0.00001)
svm_clf.fit(X_rus, y_rus)

#predict using the test data
y_pred = svm_clf.predict(X_test_standardized)

#print all the related metrics
print("test accuracy", accuracy_score(y_test,y_pred))
print("training accuracy", accuracy_score(y_rus,svm_clf.predict(X_rus)))

print("The classification report on training data is \n")
print(classification_report(y_rus, svm_clf.predict(X_rus)))
print("The classification report on test data is \n")
print(classification_report(y_test, svm_clf.predict(X_test_standardized)))

#print testing accuracy, f score, precision and recall
print(f'Accuracy: {svm_clf.score(X_test_standardized, y_test)}')
print(f'f-score: {f1_score(y_test, y_pred)}')
print(f'recall: {recall_score(y_test, y_pred)}')
print(f'precision: {precision_score(y_test, y_pred)}')


test accuracy 0.9993258616331002
training accuracy 0.8902439024390244
The classification report on training data is 

              precision    recall  f1-score   support

           0       0.82      1.00      0.90       369
           1       1.00      0.78      0.88       369

    accuracy                           0.89       738
   macro avg       0.91      0.89      0.89       738
weighted avg       0.91      0.89      0.89       738

The classification report on test data is 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     71079
           1       0.84      0.76      0.79       123

    accuracy                           1.00     71202
   macro avg       0.92      0.88      0.90     71202
weighted avg       1.00      1.00      1.00     71202

Accuracy: 0.9993258616331002
f-score: 0.7948717948717948
recall: 0.7560975609756098
precision: 0.8378378378378378
